## Iterative Adv Training

### For iter in Range(20):
-       Step 1: Generate training adv examples using weights from last epoch to 1) overwrite shot_adv.png 2) create coco_perturb_train.json
-       Step 2: Generate testing adv examples using weights from last epoch 1) overwrite shot_adv.png 2) create coco_perturb_test.json
-       Step 3: Mix downsampled normal training data(D0) with perturbed training data(Di) (4:1) and create coco_adv_mix.json
-       Step 4: Evaluate training Acc on all D0+Di examples (9 metrics)
-       Step 5: Evaluate testing Acc on clean testing 
-       Step 6: Evaluate testing Acc on perturbed testing (perturbed with weights from last epoch)

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pickle

In [2]:
import os
os.chdir('..')

In [3]:
!pwd

/home/l/liny/yizhe/code/detectron2-1


In [4]:
def evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, name):
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
        
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
        

In [ ]:

for i in range(1, 20):
    if i == 0:
        weight_last_epoch = 'output/rcnn_bet365/model_final.pth'
    else:
        weight_last_epoch = 'output/adv_%s/model_final.pth'%(str(i - 1))
    output_this_epoch = 'output/adv_%s'%(str(i))
    weight_this_epoch = output_this_epoch + '/model_final.pth'
    
    if not (os.path.exists(weight_last_epoch)):
        print('Weight for last epoch does not exist!')
        
    os.makedirs(output_this_epoch, exist_ok=True)
        
    '''generate training, testing adv examples'''
    !python run_DAG.py --test False --cfg-path configs/faster_rcnn_bet365.yaml \
                                               --weights-path {weight_last_epoch} \
                                               --results-save-path coco_instances_ignore_train.json 

    !python run_DAG.py --test True --cfg-path configs/faster_rcnn_bet365.yaml \
                                              --weights-path {weight_last_epoch} \
                                              --results-save-path coco_instances_ignore_test.json 
        
    #mix normal with training
    !python detectron2_1/mix_normal_adv.py --normal-path data/benign_data/coco_train.json \
                                           --adv-path data/benign_data/coco_perturbgt_train.json \
                                           --save-path data/benign_data/coco_adv_mix.json 

    #retrain 
    !python train_net.py --config-file configs/faster_rcnn_adv.yaml \
                                       OUTPUT_DIR {output_this_epoch} \
                                       MODEL.WEIGHTS {weight_last_epoch} 
    
    ######################## evaluate on training D0 + Di ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_train.yaml \
                                                   MODEL.WEIGHTS {weight_this_epoch} \
                                                   OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_adv_mix.json' # here the ground-truth is for the original-sized image
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' # here the prediction is for the resized image
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'train')
        
    #########################  evaluate on clean testing ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'test')

        
    #########################  evaluate on perturbed testing ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_test.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_perturbgt_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'perturb_test')



In [ ]:
with open('./output/adv_1/logo_testacc.pkl', 'rb') as f:
    logo_testacc = pickle.load(f)
    
with open('./output/adv_1/input_testacc.pkl', 'rb') as f:
    input_testacc = pickle.load(f)
    
with open('./output/adv_1/all_testacc.pkl', 'rb') as f:
    all_testacc = pickle.load(f)

In [ ]:
with open('./output/adv_1/logo_trainacc.pkl', 'rb') as f:
    logo_trainacc = pickle.load(f)
    
with open('./output/adv_1/input_trainacc.pkl', 'rb') as f:
    input_trainacc = pickle.load(f)
    
with open('./output/adv_1/all_trainacc.pkl', 'rb') as f:
    all_trainacc = pickle.load(f)

In [ ]:
logo_testacc.evaluate()
logo_testacc.accumulate()
logo_testacc.summarize()